[Home](../index.ipynb) / ADC-Wandler: Lösungen
***
<span style="font-size:20pt;">ADC-Wandler: Lösungen</span>

# Aufgaben
1) Statt der internen LED soll eine blaue LED an Pin 14 angeschlossen werden (**Vorwiderstand nicht vergessen!**).  
   Wie oben soll die LED nur leuchten, solange der Taster an Pin 25 gedrückt ist.  
   Zusätzlich soll auf dem Display der An/Aus-Zustand der LED geeignet angezeigt werden.
2) Die LED wird mit einem Tastendruck eingeschaltet und beim nächsten Tastendruck wieder ausgeschaltet.
3) Wird der Taster **lange** gedrückt, soll die LED, solange der Taster gedrückt ist, von ihrem aktuellen Helligkeitswert aus langsam zwischen Hell und Dunkel gedimmt werden.  
   Bei **kurzem** Betätigen des Tasters soll die LED wie in 2) aus- bzw. auf den letzten Helligkeitswert eingeschaltet werden.

## Aufgabe 1

In [ ]:
%serialconnect

ID_BUTTON_ONOFF = 18
ID_LED          = 25

from machine import Pin
import display
import math

display = display.Display()


btnOnOff = Pin( ID_BUTTON_ONOFF, Pin.IN, Pin.PULL_UP )
led      = Pin( ID_LED, Pin.OUT )

display.setCenter( 63, 31 )

def paintOn():
    display.fillCircle( 0, 0, 20 )
    
    for iIndex in range( 12 ):
        display.lineP( 0, 0, iIndex*math.pi/6, 20, 25 )
        
def paintOff():
    display.circle( 0, 0, 20 )
    display.line( -13, -13, 13,  13 )
    display.line( -13,  13, 13, -13 )

bIsOnLast = -1
bIsOn = -1

def paint( bIsOn ):
    global bIsOnLast
    
    if bIsOn != bIsOnLast:
        bIsOnLast = bIsOn
        
        display.clear()
        
        if bIsOn == 0 :
            paintOn()
        else :
            paintOff()
        
        display.show()


led.off()
paint( 1 )

while True:
    bIsOn = btnOnOff.value()
    
    led.on() if bIsOn == 0 else led.off()

    paint( bIsOn )


In [ ]:
## Aufgabe 2

%serialconnect

ID_BUTTON_ONOFF = 18
ID_LED          = 25

from machine import Pin
import display
import math

display = display.Display()


btnOnOff = Pin( ID_BUTTON_ONOFF, Pin.IN, Pin.PULL_UP )
led      = Pin( ID_LED, Pin.OUT )

display.setCenter( 63, 31 )

def paintOn():
    display.fillCircle( 0, 0, 20 )
    
    for iIndex in range( 12 ):
        display.lineP( 0, 0, iIndex*math.pi/6, 20, 25 )
        
def paintOff():
    display.circle( 0, 0, 20 )
    display.line( -13, -13, 13,  13 )
    display.line( -13,  13, 13, -13 )

bIsOnLast = -1
bIsOn = -1

def paint( bIsOn ):
    global bIsOnLast
    
    if bIsOn != bIsOnLast:
        bIsOnLast = bIsOn
        
        display.clear()
        
        if bIsOn == 0 :
            paintOn()
        else :
            paintOff()
        
        display.show()

        

import time

class PushButton :
    def __init__( self, _pinNr, _pressedValue = 0, _lTimeMillisWait = 20 ) :
        self._pin = Pin( _pinNr, Pin.IN, Pin.PULL_UP )
        self._pressedValue     = _pressedValue
        self._valueLast        = _pressedValue ^ 1 # self._pin.value() # start with not pressed
        self._lTimeMillisWait  = _lTimeMillisWait
        self._lTimeMillisStart = 0
        self._bWasPressed      = False
        self._isPressedState   = False
        

    def wasPressed( self ):
        self.update()
        
        if self._bWasPressed :
            self._bWasPressed = False
            return True

        else :
            return False
        
        
    def isDown( self ):
        self.update()

        return self._valueLast == self._pressedValue
        
        
    def update( self ):    
        
        if self._isPressedState :
            if time.ticks_diff(time.ticks_ms(), self._lTimeMillisStart ) > self._lTimeMillisWait:
                valueNew = self._pin.value()
                
                if valueNew == self._pressedValue :
                    if valueNew != self._valueLast :
                        self._bWasPressed = True
                else :
                    self._isPressedState = False
                
                self._valueLast = valueNew
                
        else :
            if self._pin.value() != self._valueLast :
                self._isPressedState = True
                self._lTimeMillisStart = time.ticks_ms()
                
                
btnBreak = PushButton( PIN_BUTTON )

led.off()
paint( 1 )

while True:
    if btnBreak.wasPressed():
        bIsOn = not bIsOn
    
        led.on() if bIsOn == 0 else led.off()

        paint( bIsOn )


print( "Button was pressed!" )                

In [ ]:
%serialconnect
# Warning: Timer0 is used by the functions millis() and sleep() *and* Webserver,
# or if you manually set up timer0, these functions will not work correctly.

ID_BUTTON_ONOFF = 18
ID_LED          = 25

L_DEBOUNCE_MILLIS  =  30
L_LONGPRESS_MILLIS = 200

PRESSED = 0

STATE_NOT_PRESSED = 0
STATE_PRESSED     = 1 
STATE_LONGPRESSED = 2

I_BRIGHTNESS_MIN =    0
I_BRIGHTNESS_MAX = 1000

iBrightnessOn       = I_BRIGHTNESS_MAX 
iBrightness         = I_BRIGHTNESS_MIN

lTimePreviousMillis = 0

iPushButtonState = 0
iStep = 4


from machine import PWM, Pin
import time

led      = PWM( Pin(ID_LED, Pin.OUT), freq=1000, duty=I_BRIGHTNESS_MIN)  # 25 HelTec, 13 Croduino
btnOnOff = Pin( ID_BUTTON_ONOFF, Pin.IN, Pin.PULL_UP )

 

def toggleLED():
    global iBrightness
    global iBrightnessOn
    iBrightness = iBrightnessOn if iBrightness == 0 else 0

    led.duty( iBrightness )


def dimmLED():
    global iBrightness
    global iBrightnessOn
    global iStep
    
    iBrightnessOn += iStep
    iBrightness = iBrightnessOn

    if iBrightnessOn <= I_BRIGHTNESS_MIN :
        iBrightnessOn = I_BRIGHTNESS_MIN
        iStep *= -1
        
    elif iBrightnessOn >= I_BRIGHTNESS_MAX :
        iBrightnessOn = I_BRIGHTNESS_MAX
        iStep *= -1

    led.duty( iBrightness )
    
    time.sleep_ms( 20 )    
    
    
lTimeDiff = 0

try:
    while True:
        if iPushButtonState == STATE_NOT_PRESSED :
            lTimeDiff = time.ticks_diff(time.ticks_ms(), lTimePreviousMillis)

            if lTimeDiff > L_DEBOUNCE_MILLIS :
                if btnOnOff.value() == PRESSED :
                    iPushButtonState = STATE_PRESSED
                    lTimePreviousMillis = time.ticks_ms()


        elif iPushButtonState == STATE_PRESSED :
            lTimeDiff = time.ticks_diff(time.ticks_ms(), lTimePreviousMillis)

            if lTimeDiff > L_LONGPRESS_MILLIS :
                iPushButtonState = STATE_LONGPRESSED

            elif lTimeDiff > L_DEBOUNCE_MILLIS :
                if btnOnOff.value() != PRESSED :
                    toggleLED()
                    iPushButtonState = STATE_NOT_PRESSED
                    lTimePreviousMillis = time.ticks_ms()

        elif iPushButtonState == STATE_LONGPRESSED :
            dimmLED(); 

            if  btnOnOff.value() != PRESSED :
                iPushButtonState = STATE_NOT_PRESSED
                lTimePreviousMillis = time.ticks_ms()

finally:
    led.deinit() # falls der Pin im selbem Porgramm noch mal als on/off- Pin verwendet werden soll.


Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.
.............................................